# Workflow Debugging

In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# non_zero_indices = []
# for i in range(len(X[0])):
#    if sum(X[:,i]) > 0:
#        non_zero_indices.append(i)
# X = X[:, non_zero_indices]
# openqasm_gates_list = utils.get_openqasm_gates()
# res = [openqasm_gates_list[i] for i in range(0, len(openqasm_gates_list))]
# res.append("num_qubits")
# features = list(np.sort(np.array(res)))
#features = features[non_zero_indices]
# print(features)

# scaler = preprocessing.StandardScaler().fit(X)
# X = preprocessing.normalize(X, norm='l2')
# X = scaler.transform(X)

In [ ]:
from predictor.driver import Predictor
from predictor.src import utils

import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


training_data, names_list, scores_list = Predictor.generate_trainingdata_from_qasm_files(folder_path="qasm_files/")

In [ ]:
X, y = zip(*training_data)
X = np.array(X)
y = np.array(y)
train_decision_tree_classifier(X,y, names_list, scores_list, max_depth=50);

# Parameter Gridsearch

In [ ]:
import matplotlib.pyplot as plt
from predictor.src import utils
from sklearn.tree import plot_tree
from sklearn import tree

X_new, y_new, indices = np.array(X), np.array(y), np.array(range(len(y)))
(
    X_train,
    X_test,
    y_train,
    y_test,
    indices_train,
    indices_test,
) = train_test_split(X_new, y_new, indices, test_size=0.3, random_state=42)

clf_instance = tree.DecisionTreeClassifier()



In [ ]:
from sklearn.model_selection import GridSearchCV
tree_param = [{'criterion': ['entropy', 'gini'], 
              'max_depth': [i for i in range(1,15,1)],
              'min_samples_split': [i for i in range(2,20,2)],
              'min_samples_leaf': [i for i in range(2,20,2)],
              'max_leaf_nodes': [i for i in range(2,200, 20)],
              'max_features': [i for i in range(1,45,5)]},
             ]
clf = GridSearchCV(clf_instance, tree_param, cv=5)
clf.fit(X_train, y_train)
print(clf.best_estimator_)
print(clf.best_score_)

# Decision Path Generation

In [ ]:
clf_instance.fit(X_train, y_train)
n_nodes = clf_instance.tree_.node_count
children_left = clf_instance.tree_.children_left
children_right = clf_instance.tree_.children_right
feature = clf_instance.tree_.feature
threshold = clf_instance.tree_.threshold
node_indicator = clf_instance.decision_path(X_test)
leaf_id = clf_instance.apply(X_test)

sample_id = 0
# obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
node_index = node_indicator.indices[
    node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
]

print("Rules used to predict sample {id}:\n".format(id=sample_id))
for node_id in node_index:
    # continue to the next node if it is a leaf node
    if leaf_id[sample_id] == node_id:
        continue
    # check if value of the split feature for sample 0 is below threshold
    if X_test[sample_id, feature[node_id]] <= threshold[node_id]:
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print(
        "decision node {node} : (X_test[{sample}, {feature}] = {value}) "
        "{inequality} {threshold})".format(
            node=node_id,
            sample=sample_id,
            feature=features[feature[node_id]],
            value=X_test[sample_id, feature[node_id]],
            inequality=threshold_sign,
            threshold=threshold[node_id],
        )
    )

# Other Tree Plot Types

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import plot_tree

fig = plt.figure(figsize=(23, 5))
clf = Predictor._clf


from dtreeviz.trees import dtreeviz # remember to load the package
machines = [
        "qiskit_ibm_washington",
        "qiskit_ibm_montreal",
        "qiskit_ionq",
        "qiskit_rigetti",
        "qiskit_oqc",
        "tket_ibm_washington",
        "tket_ibm_montreal",
        "tket_ionq",
        "tket_rigetti",
        "tket_oqc",
    ]
openqasm_gates_list = utils.get_openqasm_gates()
res = [openqasm_gates_list[i] for i in range(0, len(openqasm_gates_list))]
res.append("num_qubits")
features = np.sort(np.array(res))

X, y, indices = np.array(X), np.array(y), np.array(range(len(y)))
(
    X_train,
    X_test,
    y_train,
    y_test,
    indices_train,
    indices_test,
) = train_test_split(X, y, indices, test_size=0.3, random_state=42)

clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(X_train, y_train)
viz = dtreeviz(clf, 
               X_train, 
               y_train,
               target_name="target",
               feature_names=features,
               class_names=machines,
               fancy=True
)
viz.save_svg()
viz


In [ ]:
import graphviz
clf = tree.DecisionTreeClassifier(max_depth=10)
clf = clf.fit(X_train, y_train)
dot_data = tree.export_graphviz(clf, out_file=None, 
               feature_names=features,
               class_names=machines,
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph.render("graphviz")

# Debugging

In [ ]:
from mqt.bench import get_one_benchmark

In [ ]:
from pytket.extensions.qiskit import qiskit_to_tk
from qiskit.compiler import transpile
from qiskit import QuantumCircuit

In [ ]:
from pytket.qasm import circuit_from_qasm
qc = QuantumCircuit.from_qasm_file("qasm_compiled/excitedstate_small_indep_4_oqc_qiskit_opt_2_4.qasm")
for elem in qc.data:
    print(elem)

In [ ]:
for instruction, qargs, cargs in qc5.data:
    print(instruction.name, [elem.index for elem in qargs])

In [ ]:

import json

with open('rigetti_m1_calibration.json', 'r') as f:
    backend = json.load(f)

for elem in backend:
    print(elem)

In [ ]:
backend["specs"]["2Q"]

In [ ]:
import pandas
df = pandas.read_csv('ibmq_montreal_calibrations.csv')
df.head()
# if gate_type == "sx":
#     index = 10
# elif gate_type == "rz":
#     index = 11
# elif gate_type == "x":
#     index = 11
# elif gate_type == "cx":
#     index = 12
# elif gate_type == "measure":
#     index = 5


In [ ]:
tmp = '1_2'
for elem in df.loc[1][12].split(";"):
    if tmp in (elem.split(":")[0]):
        print(elem.split(":")[1])

In [ ]:
df.loc[1][12]

In [ ]:
from predictor.src import utils
import networkx as nx

In [ ]:
def get_cmap_rigetti_m1():
    """Returns a coupling map of the circular layout scheme used by Rigetti Aspen M1.

    Keyword arguments:
    """
    c_map_rigetti = []
    for j in range(5):
        for i in range(0, 7):
            c_map_rigetti.append([i + j * 8, i + 1 + j * 8])

            if i == 6:
                c_map_rigetti.append([0 + j * 8, 7 + j * 8])

        if j != 0:
            c_map_rigetti.append([j * 8 - 6, j * 8 + 5])
            c_map_rigetti.append([j * 8 - 7, j * 8 + 6])

    for j in range(5):
        m = 8 * j + 5 * 8
        for i in range(0, 7):
            c_map_rigetti.append([i + m, i + 1 + m])

            if i == 6:
                c_map_rigetti.append([0 + m, 7 + m])

        if j != 0:
            c_map_rigetti.append([m - 6, m + 5])
            c_map_rigetti.append([m - 7, m + 6])

    for n in range(5):
        c_map_rigetti.append([n * 8 + 3, n * 8 + 5 * 8])
        c_map_rigetti.append([n * 8 + 4, n * 8 + 7 + 5 * 8])



    inverted = [[item[1], item[0]] for item in c_map_rigetti]
    c_map_rigetti = c_map_rigetti + inverted

    return c_map_rigetti

In [ ]:

arch = get_cmap_rigetti_m1()
nx.draw_kamada_kawai(nx.Graph(arch), with_labels=True, font_color="white")

In [ ]:
from qiskit.transpiler import CouplingMap

In [ ]:
print(cmap)

In [ ]:
from pytket.qasm import circuit_from_qasm
from qiskit import transpile, QuantumCircuit
qc = QuantumCircuit.from_qasm_file("qasm_compiled/excitedstate_small_indep_4_ibm_washington_qiskit_opt_2_1.qasm")
cmap = CouplingMap(couplinglist=get_cmap_rigetti_m1())
transpile(qc, coupling_map=cmap)

In [ ]:
from predictor.src import utils

In [ ]:
mapping = utils.get_rigetti_qubit_dict()

In [ ]:
with open("oqc_lucy_calibration.json", "r") as f:
    backend = json.load(f)

fid2 = []
for elem in backend["twoQubitProperties"]:
    val = backend["twoQubitProperties"][elem][
            "twoQubitGateFidelity"
        ][0].get("fidelity")
    if val:
        fid2.append(val)

avg_fid2 = sum(fid2) / len(fid2)
avg_fid2

# Nachbehandlung OQC qasm files

In [ ]:
import os
directory = "qasm_compiled"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if "oqc_qiskit" in f:
        with open(f, "r") as f:
            lines = f.readlines()
        new_name = os.path.join("qasm_compiled_postprocessed", filename)
        with open(new_name, "w") as f:
            for line in lines:
                if not ("gate rzx" in line.strip("\n") or "gate ecr" in line.strip("\n")):
                    f.write(line)
                if ("gate ecr" in line.strip("\n")):
                    f.write("gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }\n")
                    f.write("gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }\n")
                    
        qc = QuantumCircuit.from_qasm_file(new_name)
        print("New qasm file for: ", new_name)
        

    elif "oqc_tket" in f:
        with open(f, "r") as f:
            lines = f.readlines()
        new_name = os.path.join("qasm_compiled_postprocessed", filename)
        with open(new_name, "w") as f:
            count = 0
            for line in lines:
                f.write(line)
                count+=1
                if count == 2:
                    f.write("gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }\n")
                    f.write("gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }\n")
        qc = QuantumCircuit.from_qasm_file(new_name)
        print("New qasm file for: ", new_name)

# Nachbehandlung: Check if File is readable at all

In [ ]:
import os
from qiskit import QuantumCircuit
directory = "qasm_compiled"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if f.endswith(".qasm"):
        try:
            qc = QuantumCircuit.from_qasm_file(f)
        except Exception as e:
            print("Fail: ", e)
            # Delete file: os.remove(f)

In [ ]:
import glob
files=glob.glob("qasm_compiled/ae_indep_2_*")
print(files)